In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# Define the RBM model
class RBM(object):
    def __init__(self, n_visible, n_hidden):
        self.W = tf.Variable(tf.random.normal([n_visible, n_hidden], 0.01))
        self.hb = tf.Variable(tf.zeros([1, n_hidden], tf.float32))
        self.vb = tf.Variable(tf.zeros([1, n_visible], tf.float32))
        
    def sample_hidden(self, x):
        h_prob = tf.nn.sigmoid(tf.matmul(x, self.W) + self.hb)
        return h_prob, tf.nn.relu(tf.sign(h_prob - tf.random.uniform(tf.shape(h_prob))))
    
    def sample_visible(self, y):
        v_prob = tf.nn.sigmoid(tf.matmul(y, tf.transpose(self.W)) + self.vb)
        return v_prob, tf.nn.relu(tf.sign(v_prob - tf.random.uniform(tf.shape(v_prob))))

# Set hyperparameters
n_visible = 784
n_hidden = 256
lr = 0.1
n_epochs = 10
batch_size = 32

# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], n_visible)
train_data = train_images.astype(np.float32) / 255.

# Split dataset into training and validation sets
val_data = train_data[-10000:]
train_data = train_data[:-10000]

# Create RBM model
rbm = RBM(n_visible, n_hidden)

# Define optimizer and loss function
optimizer = tf.optimizers.Adam(lr)
loss_fn = tf.losses.MeanSquaredError()

# Train RBM model on training dataset
for epoch in range(n_epochs):
    avg_cost = 0.0
    for i in range(0, train_data.shape[0], batch_size):
        batch = train_data[i:i+batch_size]
        with tf.GradientTape() as tape:
            hidden_prob, hidden_state = rbm.sample_hidden(batch)
            visible_prob, visible_state = rbm.sample_visible(hidden_state)
            cost = loss_fn(batch, visible_state)
        grads = tape.gradient(cost, [rbm.W, rbm.hb, rbm.vb])
        optimizer.apply_gradients(zip(grads, [rbm.W, rbm.hb, rbm.vb]))
        avg_cost += cost / (train_data.shape[0] / batch_size)
    print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    
# Evaluate RBM model on validation dataset
hidden_prob, hidden_state = rbm.sample_hidden(val_data)
visible_prob, visible_state = rbm.sample_visible(hidden_state)
val_loss = loss_fn(val_data, visible_state)
print("Validation loss:", val_loss)


2023-04-24 07:00:24.566490: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 07:00:25.425753: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2023-04-24 07:00:25.425884: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2023-04-24 07:00:25.425894: W tensorflow/com

Epoch: 0001 cost= 0.484467119
Epoch: 0002 cost= 0.484474480
Epoch: 0003 cost= 0.484556794
Epoch: 0004 cost= 0.484459102
Epoch: 0005 cost= 0.484423399
Epoch: 0006 cost= 0.484411508
Epoch: 0007 cost= 0.484471679
Epoch: 0008 cost= 0.484502405
Epoch: 0009 cost= 0.484418690
Epoch: 0010 cost= 0.484424293
Validation loss: tf.Tensor(0.4848126, shape=(), dtype=float32)
